In [22]:
import pandas as pd
import os
import prepare_data.mind as mind
import tensorflow as tf

In [4]:
epochs = 5
seed = 40
MIND_type = 'demo'
# tmpdir = TemporaryDirectory()
# data_path = tmpdir.name
data_path = os.path.join('./data', 'mind-{}'.format(MIND_type))
print(data_path)

./data\mind-demo


In [5]:
train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

In [12]:
new_headers = ['news_id',
               'category',
               'subcategory',
               'title',
               'abstract',
               'url',
               'title_entities',
               'abstract_entities']
news_data = pd.read_csv(train_news_file, sep='\t', names=new_headers)
news_data.head(5)

,news_id,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N3112,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://www.msn.com/en-us/lifestyle/lifestyler...,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
2,N12103,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://www.msn.com/en-us/health/voices/i-was-...,[],"[{""Label"": ""National Basketball Association"", ..."
3,N20460,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://www.msn.com/en-us/health/medical/how-t...,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
4,N5409,weather,weathertopstories,It's been Orlando's hottest October ever so fa...,There won't be a chill down to your bones this...,https://www.msn.com/en-us/weather/weathertopst...,"[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W...","[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W..."


In [17]:
behaviors_header = ['impression_id',
                    'user_id',
                    'time',
                    'history',
                    'impressions']

behaviors_data = pd.read_csv(train_behaviors_file, sep='\t', names=behaviors_header)

In [18]:
behaviors_data

,impression_id,user_id,time,history,impressions
0,1,U82271,11/11/2019 3:28:58 PM,N3130 N11621 N12917 N4574 N12140 N9748,N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25...
1,2,U84185,11/12/2019 10:36:47 AM,N27209 N11723 N4617 N12320 N11333 N24461 N2211...,N13089-0 N18101-0 N1248-0 N26273-0 N12770-1 N1...
2,3,U11552,11/11/2019 1:03:52 PM,N2139,N18390-0 N10537-0 N23967-1
3,4,U68381,11/11/2019 6:44:05 AM,N27420 N11621 N25416 N25457 N5124 N11751 N1175...,N15660-0 N18609-0 N2831-0 N5677-0 N19010-0 N15...
4,5,U52303,11/12/2019 3:11:52 AM,N1332 N12667,N15645-0 N7911-1
...,...,...,...,...,...
22029,22030,U69508,11/12/2019 2:50:55 AM,N19377 N5490 N19086 N16129 N2746 N21659 N17510...,N16297-0 N15645-0 N20630-0 N10602-0 N27294-0 N...
22030,22031,U79085,11/14/2019 9:52:15 AM,N27591 N21778 N17297 N9764 N13579 N22605 N2585...,N27355-0 N9809-0 N20882-0 N8787-0 N25926-0 N38...
22031,22032,U46989,11/14/2019 8:36:23 AM,N4202 N23413 N3617 N19239 N11061 N22055 N1839 ...,N13316-0 N23592-0 N10240-0 N1994-0 N26998-0 N1...
22032,22033,U49050,11/14/2019 11:58:12 AM,N17580 N12014 N3076 N10624 N27610 N6385 N4617 ...,N23982-0 N17424-0 N17157-0 N4324-0 N22216-0 N1...


In [32]:
behaviors_data[behaviors_data.user_id == 'U79085']['impressions'].values.tolist()[1].split(' ')

['N18403-0',
 'N7910-0',
 'N13997-0',
 'N25012-0',
 'N31-0',
 'N4352-0',
 'N25809-0',
 'N1894-0',
 'N14352-0',
 'N17414-0',
 'N18110-0',
 'N843-0',
 'N23139-0',
 'N5780-0',
 'N25787-0',
 'N11875-0',
 'N10178-0',
 'N20678-0',
 'N24955-0',
 'N22990-1',
 'N7090-0',
 'N18634-0',
 'N16945-0',
 'N7184-0',
 'N10552-0',
 'N3403-0',
 'N20753-0',
 'N5813-0',
 'N26154-1',
 'N26428-0',
 'N7065-0',
 'N22817-0',
 'N10328-0',
 'N5382-0',
 'N6819-0',
 'N6911-0',
 'N12340-0',
 'N5337-0',
 'N14493-0',
 'N19215-0',
 'N15417-0',
 'N24172-0',
 'N10821-0',
 'N14767-0',
 'N23650-0',
 'N16914-0',
 'N15947-0',
 'N6596-0',
 'N13985-0',
 'N25810-0',
 'N1549-0',
 'N1502-0',
 'N13203-0',
 'N12577-0',
 'N8256-0',
 'N27910-0',
 'N20820-0',
 'N4753-0',
 'N22139-0',
 'N13576-0',
 'N10550-0',
 'N469-0',
 'N21830-0',
 'N3628-0',
 'N5347-0',
 'N4894-0',
 'N27255-0',
 'N24610-0',
 'N13594-0',
 'N15037-0',
 'N27326-0',
 'N20587-0',
 'N7250-1',
 'N28236-0',
 'N28544-0']

In [31]:
behaviors_data[behaviors_data.user_id == 'U82271']['history'].values.tolist()[0].split(' ')

['N3130', 'N11621', 'N12917', 'N4574', 'N12140', 'N9748']